In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#2. specify parameters
pipeline_params={
    "X": "something"
}
step_params={
    "X": "something_else"
}
substep_params={
    "param1":"None1",
    "param2":"None2"}

In [3]:
#3 define substep interface
from sinara.substep import NotebookSubstep, default_param_values, ENV_NAME, PIPELINE_NAME, ZONE_NAME, STEP_NAME, RUN_ID, ENTITY_NAME, ENTITY_PATH, SUBSTEP_NAME

substep = NotebookSubstep(pipeline_params, step_params, substep_params, **default_param_values("params/step_params.json"))

substep.interface(
   
    inputs =
    [
        { STEP_NAME: "prev_step", ENTITY_NAME: "entityA", ZONE_NAME: "zone" },
        { STEP_NAME: "prev_step", ENTITY_NAME: "entityB" },
        { STEP_NAME: "prev_step", ENTITY_NAME: "entityC" },
        { STEP_NAME: "ext_step", ENTITY_NAME: "entityI", ENV_NAME: "prod2", PIPELINE_NAME: "other_pipeline", ZONE_NAME: "monthly_run" },
        { STEP_NAME: "ext_step2", ENTITY_NAME: "entityJ", PIPELINE_NAME: "other_pipeline2", RUN_ID: "run-00-01-01-000000" }
    ],
    outputs = 
    [
        { ENTITY_NAME: "entityF" },
        { ENTITY_NAME: "entityG" },
        { ENTITY_NAME: "entityH" }
    ],
    custom_inputs =
    [
        { ENTITY_NAME: "entityE", ENTITY_PATH: "/data/something/entityE" },
        { ENTITY_NAME: "entityD", ENTITY_PATH: "/data/something/entityD" }
    ],
    custom_outputs =
    [
        { ENTITY_NAME: "entityK", ENTITY_PATH: "/data/something/entityK" },
        { ENTITY_NAME: "entityL", ENTITY_PATH: "/data/something/entityL" }
    ],
    tmp_inputs =
    [
        { ENTITY_NAME: "entityM" },
        { ENTITY_NAME: "entityN" },
    ],
    tmp_outputs =
    [
        { ENTITY_NAME: "entityF" },
        { ENTITY_NAME: "entityG" },
        { ENTITY_NAME: "entityH" }
    ]
)

substep.print_interface_info()

substep.exit_in_visualize_mode()

**INPUTS:**


[{'user.pipeline.zone.prev_step.entityA': '/data/home/jovyan/pipeline/zone/prev_step/run-00-01-01-000000/entityA'},
 {'user.pipeline.zone.prev_step.entityB': '/data/home/jovyan/pipeline/zone/prev_step/run-00-01-01-000000/entityB'},
 {'user.pipeline.zone.prev_step.entityC': '/data/home/jovyan/pipeline/zone/prev_step/run-00-01-01-000000/entityC'},
 {'prod2.other_pipeline.monthly_run.ext_step.entityI': '/data/preprod/other_pipeline/monthly_run/ext_step/run-00-01-01-000000/entityI'},
 {'user.other_pipeline2.zone.ext_step2.entityJ': '/data/home/jovyan/other_pipeline2/zone/ext_step2/run-00-01-01-000000/entityJ'}]




**OUTPUTS:**


[{'user.pipeline.zone.step_template.entityF': '/data/home/jovyan/pipeline/zone/step_template/run-23-10-09-154554/entityF'},
 {'user.pipeline.zone.step_template.entityG': '/data/home/jovyan/pipeline/zone/step_template/run-23-10-09-154554/entityG'},
 {'user.pipeline.zone.step_template.entityH': '/data/home/jovyan/pipeline/zone/step_template/run-23-10-09-154554/entityH'}]




**CUSTOM INPUTS:**


...

[{'entityK': '/data/something/entityK'}, {'entityL': '/data/something/entityL'}]




**TMP INPUTS:**


[{'cache:user.pipeline.zone.step_template.entityM': '/data/tmp/user/pipeline/zone/step_template/run-23-10-09-154554/entityM'},
 {'cache:user.pipeline.zone.step_template.entityN': '/data/tmp/user/pipeline/zone/step_template/run-23-10-09-154554/entityN'}]




**TMP OUTPUTS:**


[{'cache:user.pipeline.zone.step_template.entityF': '/data/tmp/user/pipeline/zone/step_template/run-23-10-09-154554/entityF'},
 {'cache:user.pipeline.zone.step_template.entityG': '/data/tmp/user/pipeline/zone/step_template/run-23-10-09-154554/entityG'},
 {'cache:user.pipeline.zone.step_template.entityH': '/data/tmp/user/pipeline/zone/step_template/run-23-10-09-154554/entityH'}]




In [4]:
#4 run spark
from sinara.spark import SinaraSpark

spark = SinaraSpark.run_session(0)
SinaraSpark.ui_url()

Session is run


'http://localhost:4040'

In [5]:
#5 read inputs 
prev_step_inputs = substep.inputs(step_name="prev_step")

dfA = spark.read.parquet(prev_step_inputs.entityA).pandas_api()
dfB = spark.read.parquet(prev_step_inputs.entityB).pandas_api()
dfC = spark.read.parquet(prev_step_inputs.entityC).pandas_api()

/usr/local/spark/python/pyspark/pandas/__init__.py:49: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [6]:
#6 make something to create artifacts

dfF = dfA.to_spark()
dfG = dfB.to_spark()
dfH = dfC.to_spark()

/usr/local/spark/python/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [7]:
#7 write outputs
outputs = substep.outputs()

dfF.write.parquet(outputs.entityF, mode='overwrite')
dfG.write.parquet(outputs.entityG, mode='overwrite')
dfH.write.parquet(outputs.entityH, mode='overwrite')

custom_outputs = substep.custom_outputs()
custom_inputs = substep.custom_inputs()
tmp_inputs = substep.tmp_inputs()
tmp_outputs = substep.tmp_outputs()

In [8]:
#8 stop spark
SinaraSpark.stop_session()